## 환경설정

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
!uv add langchain langchain-openai langchain-community pypdf langchain-postgres langchain_huggingface sentence-transformers

: 

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tempfile

### PDF 로드 함수

In [3]:
loader = PyPDFLoader("data/bc_platinum.pdf")
pages = loader.load() 

total_document = "\n".join([page.page_content for page in pages])

In [4]:
total_document

"BC 플래티늄 카드 상품서비스 가이드\n01\nBC PLATINUM \nSERVICE\n당신의 가치를 만드는 카드 \nBC 플래티늄카드\n회원님과 함께 신용사회의 새 지평을 열어온 비씨카드가 오랫동안 다져온 신용카드 \n서비스 노하우를 바탕으로 회원님을 VIP 회원으로 모시게 되었습니다. \nBC 플래티늄카드는 기존의 서비스와는 차별화되는 프리미엄급 서비스만을 제시\n하여 회원님을 차원이 다른 고품격의 세계로 안내합니다. \n신규회원을 위한 가입축하 특급호텔 무료 숙박권 제공 서비스를 비롯해 플래티늄  \n카드 회원님을 위한 전용 매거진 'THE BC' 제공 서비스, 국내선 항공탑승시 동반자  \n항공권 무료 제공 서비스, 전세계 1,000여개 공항의 귀빈급 전용라운지 이용  \n서비스, 회원님의 사용내역을 바로 문자메세지로 통지하는 바로알림(SMS) 서비스, \n제주지역 렌터카 24시간 지원 서비스, 국내·외 호텔/렌터카 이용할인 및 우대  \n서비스, 여행상해, 휴일상해 및 골프상해,구매물품 분실 보상 뿐만 아니라 여행중 \n불편에 따른 여행 불편 보상보험 및 골프용품 보상보험에 이르는 다양한 종합보험 \n서비스를 제공합니다. \n또한, 저희는 플래티늄카드 이용 및 서비스 이용을 지원하기 위해 BC VIP 전용 상담 \n데스크를 운영함으로써, 좀더 가까운 밀착 서비스, 깊이있는 만족 서비스, 고품격 \n서비스 제공을 위해 최선의 노력을 다하겠습니다. \nVIP만을 위한 BC 플래티늄카드.\n삶의 가치를 아시는 귀하께 드립니다.\n상품서비스 가이드\n※ 카드 신청 전 카드 상품별 연회비, 이용조건 등에 관한 세부 사항을 상품\n    설명서, 약관 및 홈페이지 에서 확인하시기 바랍니다 \n※ 서비스 이용 시 전월실적에 따른 이용 제한, 차등할인 및 이용한도 등 서비스\n     제공 기준은 'BC 카드 홈페이지 및 카드상품 안내장’ 에서 확인하시기\n     바랍니다.\n16 문화 서비스 \n· VIP 문화공연 할인\n· KBS 교향악단 정기연주회 초대 \

In [5]:
prompt_template = """
당신은 금융 전문가입니다. 다음 문서를 읽고 사용자의 질문에 대한 답변을 제공해주세요.

<document>
{document}
</document>

<question>
{question}
</question>

** 답변 ** :
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()

result = chain.invoke({"document": total_document, "question": "VIP 콜센터 전화번호?"})

print(result)

VIP 콜센터 전화번호는 1566-7890입니다.


# 텍스트 분할(Text Splitting) 

- 대규모 텍스트 문서를 처리할 때 매우 중요한 전처리 단계
- 고려사항:
    1. 문서의 구조와 형식
    2. 원하는 청크 크기
    3. 문맥 보존의 중요도
    4. 처리 속도 


# Document Loader
[https://python.langchain.com/api_reference/community/document_loaders.html#]

1. CharacterTextSplitter
- 분할 기준: 지정된 단일 문자(예: \n, 공백 등)를 기준으로 텍스트를 분할합니다.​
- 특징:
단순한 문자 기반 분할 방식으로, 구조화되지 않은 텍스트에 적합합니다.​
문맥이나 의미를 고려하지 않기 때문에, 문장이나 단어가 중간에 잘릴 수 있습니다.

2. RecursiveCharacterTextSplitter
- 분할 기준: 여러 구분자(기본값: ["\n\n", "\n", " ", ""])를 우선순위에 따라 재귀적으로 적용하여 텍스트를 분할합니다.​
- 특징:
문단 → 문장 → 단어 순으로 분할을 시도하여, 텍스트의 의미와 문맥을 최대한 보존합니다.​
구조화된 텍스트나 자연어 처리에서 의미 단위를 유지하려는 경우에 적합합니다.

3. TokenTextSplitter
- 분할 기준: 토큰 수를 기준으로 텍스트를 분할합니다.​
- 특징: LLM의 토큰 제한을 고려하여 텍스트를 분할하므로, 모델 입력에 최적화된 형태로 텍스트를 준비할 수 있습니다.​ 언어별 토크나이저를 활용하여 정확한 토큰 단위 분할이 가능합니다


In [ ]:
from langchain_text_splitters import CharacterTextSplitter 

# 텍스트 분할기 초기화 (기본 설정값 적용 )
text_splitter = CharacterTextSplitter(

    # CharacterTextSplitter의 기본 설정값
    separator = "\n\n",         # 청크 구분자: 두 개의 개행문자
    is_separator_regex = False,  # 구분자가 정규식인지 여부

    # TextSplitter의 기본 설정값
    chunk_size = 500,          # 청크 길이
    chunk_overlap = 100,        # 청크 중첩
    length_function = len,      # 길이 함수 (문자열 길이)
    keep_separator = False,     # 구분자 유지 여부
    add_start_index = False,   # 시작 인덱스 추가 여부
    strip_whitespace = True,   # 공백 제거 여부
)

# 텍스트 분할 - split_text() 메서드 사용
texts = text_splitter.split_text(total_document)

# 분할된 텍스트 개수 출력
print(f'분할된 텍스트 개수: {len(texts)}')

# 첫 번째 분할된 텍스트 출력
print(f'첫 번째 분할된 텍스트: {texts[0]}')

print(f'--------------------------------')

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,             # 청크 크기
    chunk_overlap=100,           # 청크 중 중복되는 부분 크기
    length_function=len,         # 글자 수를 기준으로 분할
    separators=["\n\n", "\n", " ", ""],  # 구분자 - 재귀적으로 순차적으로 적용 
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pages)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200])
    print("-" * 100)
    print(chunk.page_content[-200:])
    print("=" * 100)
    print()

생성된 텍스트 청크 수: 85
각 청크의 길이: [20, 488, 491, 490, 463, 469, 205, 497, 474, 459, 459, 499, 315, 483, 473, 493, 477, 485, 482, 455, 477, 470, 343, 497, 465, 442, 458, 477, 229, 469, 493, 452, 487, 493, 460, 324, 481, 474, 491, 484, 476, 488, 364, 458, 490, 481, 495, 376, 500, 477, 460, 452, 474, 474, 492, 496, 239, 499, 482, 468, 311, 485, 472, 191, 489, 491, 493, 497, 491, 454, 479, 239, 497, 484, 495, 333, 475, 463, 490, 326, 491, 450, 497, 134, 177]

BC 플래티늄 카드 상품서비스 가이드
----------------------------------------------------------------------------------------------------
BC 플래티늄 카드 상품서비스 가이드

01
BC PLATINUM 
SERVICE
당신의 가치를 만드는 카드 
BC 플래티늄카드
회원님과 함께 신용사회의 새 지평을 열어온 비씨카드가 오랫동안 다져온 신용카드 
서비스 노하우를 바탕으로 회원님을 VIP 회원으로 모시게 되었습니다. 
BC 플래티늄카드는 기존의 서비스와는 차별화되는 프리미엄급 서비스만을 제시
하여 회원님을 차원이 다른 고품격의 세계로
----------------------------------------------------------------------------------------------------
 국내선 항공탑승시 동반자  
항공권 무료 제공 서비스, 전세계 1,000여개 공항의 귀빈급 전용라운지 이용  
서비스, 회원님의 사용내역을 바로 문자메세지로 통지하는 바로알림(SM